This is a notebook for testing different iterations of the library, new features, etc.

In [1]:
import stock_analysis as sa
from ipysheet import from_dataframe as fd

In [2]:
sa.help()

The stock analysis library includes the following functionality:
Watchlist class: a watchlist that stores stocks with their priorities
Watchlist functionality: push, front, remove, pop, view
get_metrics(ticker, metrics_to_display): display a metric for a given stock
quick_view(ticker): view various important metrics pertaining to a stock
daily_view(ticker,num_days,display=[]): view daily changes in multiples or other metrics
competitor_view(ticker,specced_comps,category,filter_revenue): view analysis on competitors
quick_valuation(ticker,ann_ebitda_growth,future_ebitda_multiple,num_yrs,share_change,cash_change,debt_change)
quick_screener_fg(revenue_growth, num_yrs): screen for fast growers at low multiples
quick_screener_bs(): screen for stocks with a strong balance sheet


In [4]:
fd(sa.get_metrics('AAPL',['grossmargin']))

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, numeric_format=None, row_end=179, row_start=0, sque…

In [4]:
fd(sa.quick_view('AAPL'))

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, numeric_format=None, row_end=7, row_start=0, squeez…

In [6]:
fd(sa.daily_view('MSFT',3))

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, numeric_format=None, row_end=2, row_start=0, squeez…

In [9]:
sa.competitor_view('AAPL',['MSFT','XOM'])

,ticker,evebitda,pe,ps,pb,1yr_eps_growth,1yr_revenue_growth,1yr_fcf_growth,5yr_avg_eps_growth,5yr_avg_revenue_growth,5yr_avg_fcf_growth,grossmargin,netmargin,ebitdamargin,eps,revenue
0,MSFT,14.7,41.400000,6.800000,9.100000,-0.346505,0.142786,0.027854,-0.097172,0.075779,0.001408,0.652000,0.150000,0.423000,2.150000,1.103600e+11
1,XOM,7.1,12.400000,1.100000,1.500000,0.053996,0.187627,0.121113,-0.137942,-0.138766,-0.297335,0.349000,0.072000,0.172000,4.880000,2.902120e+11
2,AAPL,10.0,12.600000,2.800000,7.000000,0.294778,0.158620,0.238479,0.263196,0.221485,0.242839,0.383000,0.224000,0.316000,3.000000,2.655950e+11
3,AVG,10.6,22.133333,3.566667,5.866667,0.000756,0.163011,0.129149,0.009361,0.052832,-0.017696,0.461333,0.148667,0.303667,3.343333,2.220557e+11


In [10]:
sa.quick_valuation('AAPL',0.15,10)

Current Price: 56.435
5 Yr Price Target: 80.5724692294507


In [12]:
sa.quick_screener_fg(0,3)

AAPL
AXP
BA
CAT
CSCO
CVX
DD
DIS
GE
GS
HD
IBM
INTC
JNJ
JPM
KO
MCD
MMM
MRK
MSFT
NKE
PFE


IndexError: index 0 is out of bounds for axis 0 with size 0

In [14]:
sa.quick_screener_bs()

[]

In [16]:
from stock_analysis import Watchlist

In [17]:
w = Watchlist()

In [18]:
w.front()

'STPK'

In [19]:
w.push('HITIF')

In [20]:
w.front()

'HITIF'

In [21]:
w.view()

[(1, 'HITIF'), (5, 'STPK')]

In [22]:
w.remove('STPK')

True

In [23]:
w.view()

[(1, 'HITIF')]

In [24]:
del w

In [25]:
v = Watchlist()

In [26]:
v.view()

[(1, 'HITIF')]

# Quandl testing 

In [23]:
import quandl
quandl.ApiConfig.api_key = open("quandl_key.env").read()


In [7]:
def quick_screener_fg(revenue_growth=0.15,num_yrs=5,output=False):
    SF1_df = quandl.get_table('SHARADAR/SF1')[::-1]
    tickers_df = quandl.get_table('SHARADAR/TICKERS')
    
    screened_stocks = []
    for t in SF1_df['ticker'].unique():
        #high revenue growth
        SF1_df_t = quandl.get_table('SHARADAR/SF1',ticker=t)[::-1]
        SF1_df_t['revenue_growth'] = SF1_df_t['revenue'].pct_change()
        mean_rev_growth = SF1_df_t['revenue_growth'][:num_yrs].mean()
        print(t)
        if mean_rev_growth > revenue_growth:
                #low relative valuation
                ticker_industry = tickers_df[tickers_df['ticker']==t]['industry'].values
                if ticker_industry != []:
                    competitors = tickers_df[tickers_df['industry']==ticker_industry[0]]['ticker'].values

                    mean_EBITDA_multiple = SF1_df[SF1_df['ticker'].isin(competitors)]['evebitda'].mean()
                    mean_PS_multiple = SF1_df[SF1_df['ticker'].isin(competitors)]['ps'].mean()

                    ticker_evebitda = SF1_df_t['evebitda'][0]
                    ticker_ps = SF1_df_t['ps'][0]

                    if output:
                        print("Ticker:",t)
                        print("\tEBITDA Multiple:",ticker_evebitda)
                        print("\tPS:",ticker_ps)
                        print("\tRevenue Growth:",mean_rev_growth)
                    if (ticker_evebitda < 0) or (mean_EBITDA_multiple < 0): 
                        if ticker_ps < mean_PS_multiple:
                            screened_stocks.append(t)
                    else:
                        if ticker_evebitda < mean_EBITDA_multiple:
                            screened_stocks.append(t)
    return screened_stocks 

In [8]:
quick_screener_fg(output=True)

c:\users\avyuk\appdata\local\programs\python\python36\lib\site-packages\quandl\get_table.py:38: UserWarning: To request more pages, please set paginate=True in your         quandl.get_table() call. For more information see our documentation:         https://github.com/quandl/quandl-python/blob/master/FOR_ANALYSTS.md#things-to-note
  warnings.warn(Message.WARN_PAGE_LIMIT_EXCEEDED, UserWarning)


XTGRF
XTLB


c:\users\avyuk\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  from ipykernel import kernelapp as app


XTLY
XTNT
XUE
XWES
XXIA
XXII
XYF
XYL
XYNO
Y
YAC
YALA
YAYO
YCB
YCBD
YDKN
YDLE
YDNT
YECO
YELL
YELP
YETI
YEWB
YEXT
YGEHY
YGMZ
YGTY
YGYI
YHOO
YI
YIN
YJ
YMAB
YMI
YMTX
YNDX
YOGAQ
YOKU
YONG
YORW
YPF
YPPN
YQ
YRD
YRIV
YSAC
YSG
YSYB
YTEC
YTEN
YTRA
YUANF
YUM
YUMA
YUMC
YUME
YVR
YY
ZAAP
ZAGG
ZAHLY
ZAIS
ZANE
ZARLF
ZAYO
ZAZA
ZBH
ZBRA
ZCMD
ZDGE
ZEAL
ZEN
ZEP
ZEUS
ZG
ZGNX
ZGYH
ZI
ZIGO
ZIM
ZIMCF
ZINCQ
ZION
ZIOP
ZIP
ZIPR
ZIVO
ZIXI
ZKIN
ZLAB
ZLC
ZLTQ
ZM
ZMTP
ZNGA
ZNH
ZNOG
ZNTE
ZNTL
ZOES
ZOLL
ZOLT
ZOM
ZPCM
ZPIN
ZQKSQ
ZRAN
ZS
ZSAN
ZSPH
ZSTN
ZTNO
ZTO
ZTS
ZU
ZUMZ
ZUO
ZVO
ZWRKU
ZXAIY
ZYME
ZYNE
ZYTO
ZYXI
ZZ


[]

In [9]:
td =quandl.get_table('SHARADAR/TICKERS')

In [20]:
td[td['ticker']=='ZUMZ']

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
None,,,,,,,,,,,,,,,,,,,,,


In [21]:
td

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
None,,,,,,,,,,,,,,,,,,,,,
0,SF1,196290,A,Agilent Technologies Inc,NYSE,N,Domestic Common Stock,00846U101,3826.0,Manufacturing,...,USD,California; U.S.A,2020-12-18,2014-09-26,1999-11-18,2021-02-12,1997-06-30,2020-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
1,SF1,124392,AA,Alcoa Corp,NYSE,N,Domestic Common Stock,013872106,3334.0,Manufacturing,...,USD,Pennsylvania; U.S.A,2020-10-30,2016-11-01,2016-11-01,2021-02-12,2014-12-31,2020-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
2,SF1,122827,AAAB,Admiralty Bancorp Inc,NASDAQ,Y,Domestic Common Stock,007231103,6022.0,Finance Insurance And Real Estate,...,USD,Florida; U.S.A,2019-07-29,2017-09-09,1998-09-25,2003-01-28,1997-09-30,2002-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None
3,SF1,120538,AAAGY,Altana Aktiengesellschaft,NYSE,Y,ADR Common Stock,02143N103,2834.0,Manufacturing,...,EUR,Jordan,2019-05-17,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None
4,SF1,155760,AAAP,Advanced Accelerator Applications SA,NASDAQ,Y,ADR Common Stock,00790T100,2834.0,Manufacturing,...,EUR,France,2020-10-08,2016-05-19,2015-11-11,2018-02-09,2012-12-31,2017-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,SF1,167863,NXRT,NexPoint Residential Trust Inc,NYSE,N,Domestic Common Stock,65341D102,6798.0,Finance Insurance And Real Estate,...,USD,Texas; U.S.A,2020-10-29,2016-03-22,2015-03-19,2021-02-12,2013-12-31,2020-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.nexpointliving.com
9996,SF1,195794,NXST,Nexstar Media Group Inc,NASDAQ,N,Domestic Common Stock,65336K103,4833.0,Transportation Communications Electric Gas And...,...,USD,Texas; U.S.A,2020-11-08,2014-10-24,2003-11-25,2021-02-12,2002-12-31,2020-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.nexstar.tv
9997,SF1,110837,NXTC,NextCure Inc,NASDAQ,N,Domestic Common Stock,65343E108,2834.0,Manufacturing,...,USD,Maryland; U.S.A,2020-11-08,2019-05-12,2019-05-09,2021-02-12,2017-12-31,2020-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.nextcure.com


In [24]:
len(quandl.get_table('SHARADAR/SF1')['ticker'].unique())

127

In [31]:
quandl.get_table('SHARADAR/SF1',ticker='ZM')

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
None,,,,,,,,,,,,,,,,,,,,,
0,ZM,MRY,2019-12-31,2020-01-31,2020-01-31,2020-12-04,809000.0,1.289845e+09,1.136719e+09,1.095522e+09,...,276401815.0,233641336,254298014.0,2.665,1.289845e+09,0.0,1057000.0,0.0,5.521,7.616920e+08
1,ZM,MRY,2018-12-31,2019-01-31,2019-01-31,2020-12-04,-135000.0,3.545650e+08,2.847920e+08,2.767190e+08,...,256576638.0,84483094,116005681.0,3.912,3.545650e+08,0.0,765000.0,0.0,4.197,1.243780e+08
2,ZM,MRY,2017-12-31,2018-01-31,2018-01-31,2020-12-04,-531000.0,2.150190e+08,NaN,1.831620e+08,...,NaN,78119865,78119865.0,1.939,2.150190e+08,0.0,304000.0,0.0,2.752,1.146330e+08
3,ZM,MRY,2016-12-31,2017-01-31,2017-01-31,2020-12-04,NaN,NaN,NaN,NaN,...,NaN,70309256,70309256.0,0.865,NaN,NaN,172000.0,NaN,NaN,NaN
4,ZM,MRT,2020-09-30,2020-10-31,2020-10-31,2020-12-04,1317000.0,3.050311e+09,2.258202e+09,2.624276e+09,...,284420980.0,284783006,299258765.0,6.872,3.025971e+09,0.0,881000.0,0.0,10.626,1.210328e+09
5,ZM,MRT,2020-06-30,2020-07-31,2020-07-31,2020-12-04,2772000.0,2.624941e+09,1.792638e+09,2.232102e+09,...,282102986.0,282850805,297162309.0,4.761,2.600601e+09,0.0,5821000.0,0.0,9.194,9.447760e+08
6,ZM,MRT,2020-03-31,2020-04-30,2020-04-30,2020-12-04,1887000.0,2.067709e+09,1.403884e+09,1.782957e+09,...,279001616.0,279891111,295184958.0,2.961,2.067709e+09,0.0,2841000.0,0.0,7.388,7.492290e+08
7,ZM,MRT,2019-12-31,2020-01-31,2020-01-31,2020-12-04,809000.0,1.289845e+09,1.136719e+09,1.095522e+09,...,276401815.0,233641336,254298014.0,2.665,1.289845e+09,0.0,1057000.0,0.0,5.521,7.616920e+08
8,ZM,MRT,2019-09-30,2019-10-31,2019-10-31,2020-12-04,651000.0,1.188057e+09,9.028992e+08,1.019399e+09,...,273551907.0,273316850,292771122.0,1.976,1.188057e+09,0.0,2238000.0,0.0,4.347,7.015960e+08


In [32]:
quandl.get_table('SHARADAR/SF1')['ticker']

None
0          ZZ
1          ZZ
2          ZZ
3          ZZ
4          ZZ
        ...  
9995    XTGRF
9996    XTGRF
9997    XTGRF
9998    XTGRF
9999    XTGRF
Name: ticker, Length: 10000, dtype: object

In [33]:
test_df = quandl.get_table('SHARADAR/SF1',paginate=True)

KeyboardInterrupt: 

In [38]:
quandl.get_table('SHARADAR/SF1',ticker='AAPL',dimension='MRY')[::-1][0]

KeyError: 0